In [ ]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.3 MB/

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time
from torchinfo import summary
from tabulate import tabulate

**Dataset Utilization provided by NLBSE**

In [ ]:
splits = {'java_train': 'data/java_train-00000-of-00001.parquet', 'java_test': 'data/java_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,Abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,Abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | Ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,AbstractContractGetFileStatusTest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,AbstractContractGetFileStatusTest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,AbstractContractGetFileStatusTest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,AbstractContractGetFileStatusTest.java,accept everything.,1,accept everything. | AbstractContractGetFileSt...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,AbstractContractGetFileStatusTest.java,accept nothing.,1,accept nothing. | AbstractContractGetFileStatu...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,AbstractContractGetFileStatusTest.java,equals the @code match field.,1,equals the @code match field. | AbstractContra...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,AbstractS3ACommitterFactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,ApplicationConstants.java,environment for applications.,1,environment for applications. | ApplicationCon...,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,abstractcontractgetfilestatustest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,abstractcontractgetfilestatustest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,abstractcontractgetfilestatustest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,abstractcontractgetfilestatustest.java,accept everything.,1,accept everything | abstractcontractgetfilesta...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,abstractcontractgetfilestatustest.java,accept nothing.,1,accept nothing | abstractcontractgetfilestatus...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,abstractcontractgetfilestatustest.java,equals the @code match field.,1,equals the code match field | abstractcontract...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,abstracts3acommitterfactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,applicationconstants.java,environment for applications.,1,environment for applications | applicationcons...,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
model = AutoModelForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=7)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [ ]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            # Handle the case where logits are missing, e.g., raise an exception or return a default loss
            raise ValueError("Logits are missing from the model output.")  # Or return a default loss

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# print("Best trial:")
# print(study.best_trial)

In [ ]:
# best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=9,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16 * 2,
    learning_rate=5e-05,
    weight_decay=0.0012213826960895207,
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)


trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print("<<=================Training Time================>>")
print(f"Training Time: {training_time:.2f} seconds")
print("<<==============================================>>")

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6091
  Num Epochs = 9
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1710


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.122588,0.960604,0.815836,0.802691,0.807866
1,No log,0.096691,0.967170,0.898994,0.845612,0.854489
2,0.148900,0.087719,0.970359,0.909306,0.874439,0.888783
3,0.148900,0.090959,0.970828,0.907066,0.893017,0.899733
4,0.148900,0.093731,0.972517,0.912144,0.893017,0.902004
5,0.053800,0.101455,0.971110,0.902805,0.901986,0.902142
6,0.053800,0.104274,0.970828,0.901269,0.898142,0.899641
7,0.025000,0.104716,0.971110,0.900326,0.900705,0.900332
8,0.025000,0.107143,0.971391,0.900303,0.901986,0.900962


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weigh

<<=================Training Time================>>
Training Time: 2517.67 seconds
<<==============================================>>


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32


{'eval_loss': 0.10714270919561386, 'eval_accuracy': 0.9713910514961073, 'eval_precision': 0.9003029437256498, 'eval_recall': 0.9019859064702114, 'eval_f1': 0.9009617764088442, 'eval_runtime': 20.7138, 'eval_samples_per_second': 73.526, 'eval_steps_per_second': 2.317, 'epoch': 9.0}


### Model Saving

In [ ]:
!mkdir 'best_model_codebert-base_java'
!mkdir 'best_model_codebert-base_java_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_codebert-base_java')
tokenizer.save_pretrained('./best_model_codebert-base_java_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_codebert-base_java')
tokenizer = AutoTokenizer.from_pretrained('./best_model_codebert-base_java_tokenizer')

Saving model checkpoint to ./best_model_codebert-base_java
Configuration saved in ./best_model_codebert-base_java/config.json
Model weights saved in ./best_model_codebert-base_java/pytorch_model.bin
tokenizer config file saved in ./best_model_codebert-base_java_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_codebert-base_java_tokenizer/special_tokens_map.json
loading configuration file ./best_model_codebert-base_java/config.json
Model config RobertaConfig {
  "_name_or_path": "./best_model_codebert-base_java",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_

### Model Testing

In [ ]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [ ]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [ ]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

print("<<=================Evaluation Metrics================>>")
print("Evaluation Metrics:", metrics)
print("<<==============================================>>")
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")

***** Running Prediction *****
  Num examples = 1725
  Batch size = 32


<<=================Evaluation Metrics================>>
Evaluation Metrics: {'accuracy': 0.9397101449275362, 'precision': 0.8035180736359362, 'recall': 0.8088658606793322, 'f1': 0.8029414330168881}
<<==============================================>>
<<=================Inference Time================>>
Inference Time: 23.44 seconds
<<==============================================>>


In [ ]:
!zip -r best_model_codebert-base_java.zip './best_model_codebert-base_java'
!zip -r best_model_codebert-base_java_tokenizer.zip './best_model_codebert-base_java_tokenizer'

  adding: best_model_codebert-base_java/ (stored 0%)
  adding: best_model_codebert-base_java/pytorch_model.bin (deflated 7%)
  adding: best_model_codebert-base_java/config.json (deflated 55%)
  adding: best_model_codebert-base_java/training_args.bin (deflated 51%)
  adding: best_model_codebert-base_java_tokenizer/ (stored 0%)
  adding: best_model_codebert-base_java_tokenizer/special_tokens_map.json (deflated 51%)
  adding: best_model_codebert-base_java_tokenizer/tokenizer_config.json (deflated 73%)
  adding: best_model_codebert-base_java_tokenizer/vocab.json (deflated 59%)
  adding: best_model_codebert-base_java_tokenizer/tokenizer.json (deflated 82%)
  adding: best_model_codebert-base_java_tokenizer/merges.txt (deflated 53%)


In [ ]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_codebert-base_java.zip "/content/drive/MyDrive/nlbse/models/codebert-base_java/"
!cp best_model_codebert-base_java_tokenizer.zip "/content/drive/MyDrive/nlbse/models/codebert-base_java_Tokenizer/"

ValueError: mount failed

### Load and Test Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/nlbse/models/codebert-base_java/'
tokenizer_folder = '/content/drive/MyDrive/nlbse/models/codebert-base_java_Tokenizer/'

!unzip '/content/drive/MyDrive/nlbse/models/codebert-base_java/best_model_codebert-base_java.zip' -d './'
!unzip '/content/drive/MyDrive/nlbse/models/codebert-base_java_Tokenizer/best_model_codebert-base_java_tokenizer.zip' -d './'

In [ ]:
best_model_codebert_base_java = 'best_model_codebert-base_java'
best_model_codebert_base_java_tokenizer = 'best_model_codebert-base_java_tokenizer'

In [ ]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_codebert_base_java)
tokenizer = AutoTokenizer.from_pretrained(best_model_codebert_base_java_tokenizer)

In [ ]:
text = "for detecting automatically generated fields."
start_time = time.time()
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
end_time = time.time()
inference_time = end_time - start_time
print("===============Inference Time===============")
print(f"Inference Time: {inference_time:.2f} seconds")
print("============================================")
output

In [ ]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")